In [12]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d')

In [13]:
ID=timestamp+"_robustness"
ID

'20220226_robustness'

In [11]:
rm -rf /data/output/$ID

In [14]:
mkdir /data/output/$ID

In [15]:
cp meta.csv /data/output/$ID

In [17]:
!cd onnxzoo2filterdb && python3 create_filter_storage.py /ssd1/robustness_onnx_zoo/ /data/output/$ID/meta.csv /data/output/$ID/ --split_after 1000 --force

100%|█████████████████████████████████████████| 119/119 [02:02<00:00,  1.03s/it]


In [18]:
!cd onnxzoo2filterdb && python3 extract_filters.py /data/output/$ID/ /data/output/$ID/datastorage.meta.csv /data/output/$ID/filterinfo.csv /data/output/$ID/

2022-02-26 19:10:21,632 [INFO]  starting extraction Namespace(dir='/data/output/20220226_robustness/', filter_info_output_file='/data/output/20220226_robustness/filterinfo.csv', filter_output_file='/data/output/20220226_robustness/', filter_shape=(3, 3), first_convs_only=False, meta_file=<_io.TextIOWrapper name='/data/output/20220226_robustness/datastorage.meta.csv' mode='r' encoding='UTF-8'>)
2022-02-26 19:10:21,636 [INFO]  loading chunk /data/output/20220226_robustness/datastorage.0000.pkl
2022-02-26 19:10:39,271 [INFO]  loaded ... processing
2022-02-26 19:10:58,549 [INFO]  finished chunk /data/output/20220226_robustness/datastorage.0000.pkl with 119 models
2022-02-26 19:11:00,620 [INFO]  writing filter info dataframe: /data/output/20220226_robustness/filterinfo.csv
2022-02-26 19:11:00,637 [INFO]  writing filters /data/output/20220226_robustness/
2022-02-26 19:11:51,516 [INFO]  finished


In [19]:
!echo "Created by processing_pipeline.ipynb on" $timestamp >> /data/output/$ID/logs/pipeline_log.txt

In [20]:
from onnxzoo2filterdb.filter_dataset import FilterDataset
import pandas as pd
import numpy as np
import os

root_path = "/data/output"
dataset_id = ID
filter_dataset = FilterDataset(dataset_id, root=root_path, filters=False)
fid_by_mid_with_meta = filter_dataset.df_filter_info.groupby([filter_dataset.df_filter_info.index, filter_dataset.df_filter_info.conv_depth, filter_dataset.df_filter_info.conv_depth_norm, filter_dataset.df_filter_info.layer_id]).filter_ids.apply(lambda x: np.hstack(x)).to_frame().join(filter_dataset.df_fused_meta, on=["model_id"])
fid_by_mid_with_meta["filter_ids"] = fid_by_mid_with_meta["filter_ids"].apply(lambda x: f"{x[0]}:{x[-1]}")
fid_by_mid_with_meta.to_hdf(os.path.join(root_path, dataset_id, "dataset.h5"), "meta")

Loading tables ...
 df_filter_info
 df_meta
 df_meta_spreadsheets
 merging meta
Done


/tmp/ipykernel_91636/73617082.py:11: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['filter_ids', 'model', 'path', 'producer', 'Name', 'Paper',
       'Tracer Warning', 'Selection', 'Robust', 'Network', 'Backbone',
       'Framework', 'Pretraining-Dataset', 'Training-Dataset',
       'Visual Category', 'Visual Category_micro', 'Precision', 'Task'],
      dtype='object')]

  fid_by_mid_with_meta.to_hdf(os.path.join(root_path, dataset_id, "dataset.h5"), "meta")
